In [34]:
import os

In [35]:
pwd

"E:\\Father's_Document\\code\\MlOps_project\\End_to_End_wine_price_prediction"

In [36]:
os.chdir(f"E:\\Father's_Document\\code\\MlOps_project\\End_to_End_wine_price_prediction")

In [37]:
%pwd

"E:\\Father's_Document\\code\\MlOps_project\\End_to_End_wine_price_prediction"

In [38]:
import pandas as pd
import numpy as np

In [39]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: str
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [40]:
from src.wine_qulity_predction.constants import *
from src.wine_qulity_predction.utils.common import  *

In [41]:
class ConfigurationManager :
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) ->ModelTrainingConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET


        create_directories([config.root_dir])

        model_training_config=ModelTrainingConfig(root_dir=config.root_dir,
                                                  train_data_path= config.train_data_path,
                                                  test_data_path= config.test_data_path,
                                                  model_name= config.model_name,
                                                  alpha= params.alpha,
                                                  l1_ratio= params.l1_ratio,
                                                  target_column= schema.name,)
        return model_training_config



In [42]:
from sklearn.linear_model import  ElasticNet
import  joblib

In [43]:
class ModelTrainer :
    def __init__(self,config:ModelTrainingConfig):
        self.config=config

    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)

        train_x=train_data.drop([self.config.target_column],axis=1)
        train_y=train_data[[self.config.target_column]]

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        lr=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))





In [44]:
try:
    config=ConfigurationManager()
    model_training_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_training_config)
    model_trainer.train()

except Exception as e:
    raise e

[2025-06-30 20:02:50,887 : INFO : common : yaml file : config\config.yml loadded successfully] 
[2025-06-30 20:02:50,892 : INFO : common : yaml file : params.yml loadded successfully] 
[2025-06-30 20:02:50,894 : INFO : common : yaml file : schema.yml loadded successfully] 
[2025-06-30 20:02:50,894 : INFO : common : Created directory at artifacts] 
[2025-06-30 20:02:50,895 : INFO : common : Created directory at artifacts/model_trainer] 
